In [66]:
import requests
from bs4 import BeautifulSoup
import json

1. Get class data

In [67]:
# Get all the class links
url = "https://catalog.upenn.edu/courses/"

response = requests.get(url)
header = "https://catalog.upenn.edu"
course_links = []

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    course_div = soup.find_all('div', class_='az_sitemap')

    course_uls = course_div[0].find_all('ul', class_=False)

    # Get and store all the course links
    for course_ul in course_uls:
        course_lis = course_ul.find_all('li', class_=False)
        for course_li in course_lis:
            a_elements = course_li.find_all('a')
            href_value = a_elements[0].get('href')
            course_links.append(header + href_value)
else:
    print("Failed to retrieve the page. Status code:", response.status_code)

In [72]:
def get_course_number(url):
    response = requests.get(url)
    course_numbers = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        course_listings = soup.find_all('div', class_='courseblock')

        for course in course_listings:
            p_tags = course.find_all('p')
            
            course_info = p_tags[0].text
            dept = course_info.split('\xa0')[0]
            course_number = course_info.split('\xa0')[1].split(' ')[0]
            
            course_numbers.append(dept + "-" + course_number)
    else:
        print("Failed to retrieve the page. Status code:", response.status_code)    
    
    return course_numbers

In [73]:
all_course_numbers = []

for course_link in course_links:
    course_numbers = get_course_number(course_link)
    all_course_numbers.extend(course_numbers)

In [ ]:
cookies = {
    'csrftoken': '7uoFMfLL1lLpZ9raoHvGJ1BrzWE8udxhLANeRSfEMa4XasPTt5Y3MNBd3vyX9nXH',
    'sessionid': '23ivwpc2mtrobpkjy4tdzjsr49wq5mqw',
}

headers = {
    'authority': 'penncoursereview.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'accept-language': 'en-US,en;q=0.9',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Brave";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'sec-gpc': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}

In [91]:
def extract_info(json_data):
    instructors = json_data.get('instructors', {})

    # Initialize variables to store the highest rated instructor and their quality
    highest_rInstructorQuality = -1  # Initialize to a low value
    difficulty, course_quality, work_required = None, None, None

    # Iterate through the instructors
    for instructor_id, instructor_info in instructors.items():
        rInstructorQuality = instructor_info.get('average_reviews', {}).get('rInstructorQuality', -1)
        if rInstructorQuality > highest_rInstructorQuality:
            highest_rInstructorQuality = rInstructorQuality
            difficulty = instructor_info.get('average_reviews', {}).get('rDifficulty', None)
            course_quality = instructor_info.get('average_reviews', {}).get('rCourseQuality', None)
            work_required = instructor_info.get('average_reviews', {}).get('rWorkRequired', None)

    return difficulty, course_quality, work_required

In [ ]:
# for course_number in all_course_numbers:
#     print(course_number)
#     response = requests.get(f'https://penncoursereview.com/api/review/course/{course_number}?format=json&token=platform', cookies=cookies, headers=headers)

#     if response.status_code == 200:
#         data = json.loads(response.text)

#         # Find the best instructor
#         info_arr = extract_info(data)
#     else:
#         continue
#     print(response.status_code)
#     print(response.text)

In [ ]:
# response = requests.get(f'https://penncoursereview.com/api/review/course/AFRC-1780?format=json&token=platform', cookies=cookies, headers=headers)
# data = json.loads(response.text)
# print(data)

In [110]:
course_review = {}
timeout_seconds = 1

for course_number in all_course_numbers:
    try:
        response = requests.get(f'https://penncoursereview.com/api/review/course/{course_number}?format=json&token=platform', cookies=cookies, headers=headers, timeout=timeout_seconds)

        if response.status_code == 200:
            data = json.loads(response.text)

            # Find the best instructor
            course_review[course_number] = extract_info(data)
        else:
            course_review[course_number] = []
    except requests.exceptions.Timeout:
        print(f"Request for course {course_number} timed out after {timeout_seconds} seconds.")
        course_review[course_number] = []

Request for course AFRC-1780 timed out after 3 seconds.
Request for course AFRC-2251 timed out after 3 seconds.
Request for course AFRC-2430 timed out after 3 seconds.
Request for course AFRC-4000 timed out after 3 seconds.
Request for course AFRC-5330 timed out after 3 seconds.


KeyboardInterrupt: 

In [104]:
len(course_review)

90

In [ ]:
choose the instructor rating that has the highest instructor qualirty